In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys
from pathlib import Path

sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf

In [ ]:
from choice_learn.datasets import load_electricity
elec_dataset = load_electricity(as_frame=False)

In [ ]:
from choice_learn.models.simple_mnl import SimpleMNL
from choice_learn.models.base_model import BaseLatentClassModel
from choice_learn.models.latent_class_mnl import LatentClassSimpleMNL

In [ ]:
lc_model = LatentClassSimpleMNL(n_latent_classes=3, fit_method="mle", optimizer="lbfgs", epochs=100, tolerance=1e-8)

In [ ]:
lc_model.fit(elec_dataset)

In [ ]:
lc_model.latent_logits, lc_model.models[0].weights, lc_model.models[1].weights, lc_model.models[2].weights

In [ ]:
plt.plot(hist)

In [ ]:
lc_model.evaluate(elec_dataset) * len(elec_dataset)

In [ ]:
mm = BaseMixtureModel(model_class=SimpleMNL, model_parameters={"add_exit_choice": False}, n_latent_classes=2, fit_method="em", epochs=2)

In [ ]:
mm = BaseMixtureModel(model_class=SimpleMNL, model_parameters={"add_exit_choice": False}, n_latent_classes=2, fit_method="em", epochs=2)
mm.instantiate()
print(mm.latent_logits)
mm.models[0].fit(dataset, epochs=1)
mm.models[1].fit(dataset, epochs=1)

In [ ]:
mm.tolerance = 1e-2
h = mm._fit_with_lbfgs(dataset[:25], epochs=100, verbose=1)

In [ ]:
mm.evaluate(dataset[:25]) * 25 

In [ ]:
mm.models[1].weights

In [ ]:
mm.models[0].weights

In [ ]:
mm.latent_logits

In [ ]:
mm2 = BaseMixtureModel(model_class=SimpleMNL, model_parameters={"add_exit_choice": False}, n_latent_classes=2, fit_method="em", epochs=2)
mm2.instantiate()
print(mm2.latent_logits)
mm2.models[0].fit(dataset, epochs=1)
mm2.models[1].fit(dataset, epochs=1)

In [ ]:
mm2.models[0].weights
w1 = [-1.97818, 1.25229, 18.59170, 17.22774, -9.46991, -6.34042]
w2 = [-1.47861, -0.56208, 2.48494, 2.18748, -14.88065, -20.46972 ]

In [ ]:
mm2.latent_logits = tf.constant([0.08077])
mm2.models[0].weights = [tf.constant([-1.97818, 1.25229, 18.59170, 17.22774, -9.46991, -6.34042])]
mm2.models[1].weights = [tf.constant([-1.47861, -0.56208, 2.48494, 2.18748, -14.88065, -20.46972 ])]

In [ ]:
mm2.evaluate(dataset[:25]) * 25

In [ ]:
for batch in dataset[:25].iter_batch(batch_size=-1):
    c, d = mm2.batch_predict(*batch)
    e, f = mm.batch_predict(*batch)

In [ ]:
d = [d[i][dataset.choices[i]] for i in range(25)]
f = [f[i][dataset.choices[i]] for i in range(25)]

In [ ]:
np.sum(d), np.sum(f)

In [ ]:
plt.plot(d, label="gmnl")
plt.plot(f, label="mine")
plt.legend()

In [ ]:
PPP1 = []
PPP2 = []
for i in range(25):
    print(dataset.choices[i], d[i], d[i][dataset.choices[i]])
    PPP1.append(d[i][dataset.choices[i]])
    PPP2.append(d[i][dataset.choices[i]])

In [ ]:
np.sum(np.log(PPP1)), np.sum(np.log(PPP))

In [ ]:
mm.weights = tf.concat([mm.models[0].weights, mm.models[1].weights, mm.latent_logits], axis=0)

In [ ]:
mm.tolerance = 1e-2

In [ ]:
mm.tolerance = 1e-2
mm._fit_with_lbfgs(dataset)

In [ ]:
mm.evaluate(dataset)

In [ ]:
_f

In [ ]:
_

In [ ]:
_2

In [ ]:
_f = [p[0]*_[i] + p[1]*_2[i] for i in range(4)]

In [ ]:
model = SimpleMNL(optimizer="lbfgs", epochs=100, tolerance=1e-4)
h = model.fit(dataset, sample_weight=www)
h[-1]

In [ ]:
import tensorflow as tf
model.weights = tf.constant([[-0.622498214, -0.108463198, 1.46465945, 1.02686298, -5.48458862, -5.86176395]])

In [ ]:
bigP = model.predict_probas(dataset).numpy()
p = [(bigP[i][dataset.choices[i]]) for i in range(len(dataset))]

In [ ]:
np.min(p)

In [ ]:
from choice_learn.models import ConditionalMNL
from choice_learn.models.base_model import BaseMixtureModel

In [ ]:
mix = BaseMixtureModel(model_class=SimpleMNL,
                          model_parameters={"optimizer": "lbfgs", "epochs": 100, "tolerance": 1e-2},
                          n_latent_classes=2, epochs=20, fit_method="em")

In [ ]:
mix.instantiate()
h1, h2 = mix._em_fit(dataset[:750])

In [ ]:
plt.plot(h2)

In [ ]:
(mix.models[0].weights,
mix.models[1].weights)

In [ ]:
xxx.shape

In [ ]:
w1 = [-0.4458, -0.1847, 1.2144, 0.9641, -3.2184, -3.4865]
w2 = [-0.8431, -0.1242, 1.6445, 1.4139, 1.4139, -9.2647]
p = [np.exp(0), np.exp(-0.2200)]
p = [p[i] / np.sum(p) for i in range(2)]
xxx = dataset.contexts_items_features[0][:750]

In [ ]:
p3 = []
for i in range(len(xxx)):
    c = []
    for j in range(4):
        c.append(np.sum(xxx[i][j] * w1))
    p3.append(np.array(c) / np.sum(c))
p4 = []
for i in range(len(xxx)):
    c = []
    for j in range(4):
        c.append(np.sum(xxx[i][j] * w2))
    p4.append(np.array(c) / np.sum(c))

ppp = p[0] * np.stack(p3) + p[1] * np.stack(p4)
ppp = [ppp[i][dataset.choices[i]] for i in range(750)]

In [ ]:
ppp = p[0] * np.stack(p3) + p[1] * np.stack(p4)
ppp = [ppp[i][dataset.choices[i]] for i in range(750)]

In [ ]:
np.sum(np.log(ppp))

In [ ]:
p3[0], p1[0]

In [ ]:
p1 = np.exp(np.dot(xxx, w1)) / np.sum(np.exp(np.dot(xxx, w1)), axis=1, keepdims=True)
p2 = np.exp(np.dot(xxx, w2)) / np.sum(np.exp(np.dot(xxx, w2)), axis=1, keepdims=True)

ppp = p[0] * p1 + p[1] * p2
ppp = [ppp[i][dataset.choices[i]] for i in range(750)]

In [ ]:
np.sum(np.log(ppp))

In [ ]:
np.log(np.e)

In [ ]:
(mix.models[0].weights,
mix.models[1].weights,
mix.models[2].weights)

In [ ]:
mix.weights

In [ ]:
mix.latent_logits

In [ ]:
h2

In [ ]:
plt.plot(h2)

In [ ]:
len(dataset)

In [ ]:
w1 = [-0.436786, -0.024613, 2.510545, 1.649299, -2.726110, -3.720022]
w2 = [-0.711346, -0.534724, 0.632174, 0.558454, -5.994899, -5.855283]
w3 = [-0.785066, -0.055608, 1.559850, 1.219109, -8.866931, -8.332032]

lp = [np.exp(0), np.exp(-0.515807), np.exp(0.152700)]
lp = np.array(lp) / np.sum(lp)

In [ ]:
mmm = [SimpleMNL(optimizer="lbfgs", epochs=100, tolerance=1e-4) for _ in range(3)]
for _ in mmm:
    _.fit(dataset)

In [ ]:
mmm[0].weights = tf.constant(w1)
mmm[1].weights = tf.constant(w2)
mmm[2].weights = tf.constant(w3)

In [ ]:
probas = [_.predict_probas(dataset).numpy() for _ in mmm]
c_probas = []
for prob in probas:
    prob = [prob[i][dataset.choices[i]] for i in range(len(dataset))]
    c_probas.append(prob)

class_logits = [0, -0.515807, 0.152700]
class_p = [np.exp(l) for l in class_logits]
class_p = np.array(class_p) / np.sum(class_p)
c_probas = np.sum([np.array(c_probas[i]) * class_p[i] for i in range(3)], axis=0)

In [ ]:
np.sum(np.log(c_probas))

In [ ]:
class_p

In [ ]:
np.exp(0), np.exp(-0.515807), np.exp(0.152700)

In [ ]:
probas

In [ ]:
xxx = dataset.contexts_items_features[0]

In [ ]:
p1 = np.exp(np.dot(xxx, w1)) / np.sum(np.exp(np.dot(xxx, w1)), axis=1, keepdims=True)
p2 = np.exp(np.dot(xxx, w2)) / np.sum(np.exp(np.dot(xxx, w2)), axis=1, keepdims=True)
p3 = np.exp(np.dot(xxx, w3)) / np.sum(np.exp(np.dot(xxx, w3)), axis=1, keepdims=True)

In [ ]:
p = np.sum([lp[0] * p1, lp[1] * p2, lp[2] * p3], axis=0)

In [ ]:
nll = [np.log(p[i][z]) for i, z in zip(range(len(p)), dataset.choices)]

In [ ]:
np.sum(nll)

In [ ]:
import tensorflow as tf

tf.keras.metrics.categorical_crossentropy(tf.one_hot(dataset.choices, 4), p)

In [ ]:
tf.reduce_sum(tf.keras.metrics.categorical_crossentropy(tf.one_hot(dataset.choices, 4), p))

In [ ]:
nll[:4]

In [ ]:
np.log(.1), p[0], np.log(p[0][3])

In [ ]:
df.head()